In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('./')
import json
import os
import qiskit
from qiskit import QuantumCircuit
import numpy as np
from dataclasses import dataclass
from enum import auto, Enum
import matplotlib.pyplot as plt
import math
import pandas as pd
from qiskit.visualization import plot_error_map
from component.a_backend.fake_backend import *
from mqt.bench.targets import get_available_gateset_names, get_available_device_names
from mqt.bench import BenchmarkLevel, get_benchmark
from mqt.bench.targets import get_device, get_target_for_gateset
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler import generate_preset_pass_manager
 
from qiskit.circuit import QuantumCircuit
from qiskit.visualization import plot_histogram
from component.sup_sys.backend_loader import load_backends
from qiskit.visualization import plot_error_map
from component.c_circuit_work.knitting.width_k import *
from qiskit.transpiler import *

In [38]:
qc_algorithmic_level1 = get_benchmark(benchmark="bv", level=BenchmarkLevel.ALG, circuit_size=3)
backend0 = FakeAthensV2()
transpile11 = transpile(qc_algorithmic_level1, backend=backend0)

result11 = backend0.run(transpile11, shots=1000).result()
print(result11)
counts11 = result11.get_counts(transpile11)
print(counts11)
plot1 = plot_histogram(counts11)
plot1.savefig('img/plothistogram.png')

Result(backend_name='aer_simulator', backend_version='0.17.0', qobj_id='', job_id='beeee33c-98cb-434c-9865-ae53ddc93c92', success=True, results=[ExperimentResult(shots=1000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x3': 4, '0x0': 23, '0x2': 973}), header=QobjExperimentHeader(creg_sizes=[['c', 2]], global_phase=1.5707963267949, memory_slots=2, n_qubits=5, name='bv', qreg_sizes=[['q', 5]], metadata={}), status=DONE, seed_simulator=1725201286, metadata={'time_taken': 0.007655244, 'num_bind_params': 1, 'parallel_state_update': 12, 'parallel_shots': 1, 'required_memory_mb': 1, 'input_qubit_map': [[3, 2], [1, 1], [0, 0]], 'method': 'density_matrix', 'device': 'CPU', 'num_qubits': 3, 'sample_measure_time': 0.005948074, 'active_input_qubits': [0, 1, 3], 'num_clbits': 2, 'remapped_qubits': True, 'runtime_parameter_bind': False, 'max_memory_mb': 15333, 'noise': 'superop', 'measure_sampling': True, 'batched_shots_optimization': False, 'fusion': {'applied': False, 'max_fused

In [4]:
transpile11 = transpile(qc_algorithmic_level1)
plot2 = transpile11.draw("mpl")
plot2.savefig('img/dj_algorithmic_level_3qubit_decompose.pdf')

In [5]:
qc_algorithmic_level2 = get_benchmark(benchmark="half_adder", level=BenchmarkLevel.ALG, circuit_size=3)
plot3 = qc_algorithmic_level2.draw("mpl")
plot3.savefig('img/half_adder_algorithmic_level_3qubit.pdf')

In [6]:
transpile21 = transpile(qc_algorithmic_level2)
plot4 = transpile21.draw("mpl")
plot4.savefig('img/half_adder_algorithmic_level_3qubit_decompose.pdf')
